In [1]:
import psi4
import numpy as np
from P4toC4_aux import *

In [2]:
#  C4: STO-3G  -74.963 319 056 386 7

psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", "cc-pVDZ")
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

# O
# H 1 0.96
# H 1 0.96 2 104.5


h2o = psi4.geometry("""
0 1
O            0.000000000000     0.000000000000    -0.065775570547
H            0.000000000000    -0.759061990794     0.521953018286
H            0.000000000000     0.759061990794     0.521953018286
symmetry c1
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-76.02665366131474

In [3]:
basis_mapping(wf.basisset(), verbose=2)

Atom 0
  Shell 0  s 0
  Shell 1  s 0
  Shell 2  s 0
  Shell 3  p 1
  Shell 4  p 1
  Shell 5  d 2
shell_offset[l,m]:
[[ 0  0  0  0  0  0  0  0  0]
 [ 3  5  7  0  0  0  0  0  0]
 [ 9 10 11 12 13  0  0  0  0]
 [14 14 14 14 14 14 14  0  0]
 [14 14 14 14 14 14 14 14 14]
 [ 0  0  0  0  0  0  0  0  0]]
l = 0, nl[l]=0
  m=0 -> m=0   ptr=0
l = 0, nl[l]=1
  m=0 -> m=0   ptr=1
l = 0, nl[l]=2
  m=0 -> m=0   ptr=2
l = 1, nl[l]=0
  m=0 -> m=2   ptr=7
  m=1 -> m=1   ptr=5
  m=2 -> m=0   ptr=3
l = 1, nl[l]=1
  m=0 -> m=2   ptr=8
  m=1 -> m=1   ptr=6
  m=2 -> m=0   ptr=4
l = 2, nl[l]=0
  m=0 -> m=0   ptr=9
  m=1 -> m=1   ptr=10
  m=2 -> m=2   ptr=11
  m=3 -> m=3   ptr=12
  m=4 -> m=4   ptr=13
Atom 1
  Shell 0  s 0
  Shell 1  s 0
  Shell 2  p 1
shell_offset[l,m]:
[[0 0 0 0 0 0 0 0 0]
 [2 3 4 0 0 0 0 0 0]
 [5 5 5 5 5 0 0 0 0]
 [5 5 5 5 5 5 5 0 0]
 [5 5 5 5 5 5 5 5 5]
 [0 0 0 0 0 0 0 0 0]]
l = 0, nl[l]=0
  m=0 -> m=0   ptr=14
l = 0, nl[l]=1
  m=0 -> m=0   ptr=15
l = 1, nl[l]=0
  m=0 -> m=2   ptr=18
  m=1 

array([ 0,  1,  2,  7,  5,  3,  8,  6,  4,  9, 10, 11, 12, 13, 14, 15, 18,
       17, 16, 19, 20, 23, 22, 21])

In [3]:
# basis=wf.basisset()
# print(basis.genbas())
# basis.output('')

In [4]:
n_el = wf.nalpha()
eps = np.array(wf.epsilon_a())
for i in range(n_el-3,n_el+2):
    print(f' {i+1:3d}  {eps[i]:12.7f} {eps[i]*27.21138:10.6f} eV')

   3    -0.6977965 -18.988007 eV
   4    -0.5660912 -15.404123 eV
   5    -0.4929537 -13.413952 eV
   6     0.1851020   5.036880 eV
   7     0.2558498   6.962026 eV


In [5]:
# Psi4
C=np.array(wf.Ca())
print('Psi4 MOs:', wf.nmo())
# Cfour
norb, cs = read_oldmos('OLDMOS.H2O.PVDZ', verbose=1)

Psi4 MOs: 24
reading orbitals from OLDMOS.H2O.PVDZ
  144 lines, 4 orbitals at the bottom
  nAOs = nMOs = 24


In [14]:
mo1,mo2=1,5 # valence
ao1,ao2=0,3 # s
print(np.round( C[ao1:ao2,mo1:mo2],4))
print(np.round(cs[ao1:ao2,mo1:mo2],4))

[[ 0.0081  0.      0.0022  0.    ]
 [-0.4432 -0.     -0.1502 -0.    ]
 [-0.3776 -0.     -0.3489 -0.    ]]
[[ 0.0081  0.      0.0022 -0.    ]
 [-0.4432  0.     -0.1502  0.    ]
 [-0.3776 -0.     -0.3489  0.    ]]


In [23]:
mo=6
ao1,ao2=3,9  # O-p for PVDZ
print('    Cfour        Psi4')
for ao in range(ao1,ao2):
    print(f'{ao:3d} {cs[ao,mo]:10.6f} {C[ao,mo]:10.6f}')

    Cfour        Psi4
  3   0.000000   0.000000
  4   0.000000   0.000000
  5   0.283977   0.283978
  6   0.670910   0.000000
  7  -0.000000   0.000000
  8  -0.000000   0.670912


In [8]:
C2 = psi4_to_c4(C,offset)
np.round(C2,2)

array([[ 0.99,  0.23, -0.  ,  0.1 , -0.  ,  0.13,  0.  ],
       [ 0.03, -0.83,  0.  , -0.54,  0.  , -0.88, -0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.61,  0.  ,  0.  , -0.  ,  0.99],
       [ 0.  , -0.13, -0.  ,  0.78, -0.  , -0.74, -0.  ],
       [-0.01, -0.16, -0.45,  0.28, -0.  ,  0.8 ,  0.84],
       [-0.01, -0.16,  0.45,  0.28, -0.  ,  0.8 , -0.84]])

In [9]:
write_oldmos('PSIMOS', C2)